In [5]:
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
import random
import face_recognition
import time

In [6]:
DETECT_CONF_THRESH = 0.5
RECOG_CONF_THRESH = 0.5
image_folder = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/datasets/caltech_faces_test/"
detector_main_folder = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/face_detection_model/"
encodings_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/output/caltech_train_encodings.pickle"
recognizer_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/output/caltech_train_recognizer.pickle"
label_encoder_save_path = "D:/Projects/security_camera_face_recognition/Security_Camera_Face_Recognition/face_recognition/DL_dlib_SVM/output/caltech_train_label_encoder.pickle"


In [7]:
# Load face detector model
print("[INFO] loading face detector...")
protoPath = os.path.join(detector_main_folder, "deploy.prototxt")
modelPath = os.path.join(detector_main_folder,"res10_300x300_ssd_iter_140000.caffemodel")
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
print("[INFO] face detector loaded.")
# load the actual face recognition model along with the label encoder
print("[INFO] loading trained face recognizer model...")
recognizer = pickle.loads(open(recognizer_save_path, "rb").read())
le = pickle.loads(open(label_encoder_save_path, "rb").read())



[INFO] loading face detector...
[INFO] face detector loaded.
[INFO] loading trained face recognizer model...


In [8]:
# load the image, resize it to have a width of 600 pixels (while
# maintaining the aspect ratio), and then grab the image dimensions
all_images = os.listdir(image_folder)
print("Total images in the specified folder: ", len(all_images))
for i in range(10):
    time_1 = time.time()
    random_image = random.choice(all_images)
    true_name = random_image.split()[0]
    image_path = os.path.join(image_folder, random_image)
    image = cv2.imread(image_path)

    image = imutils.resize(image, width=600)
    (h, w) = image.shape[:2]
    
    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)
    
    # apply OpenCV's deep learning-based face detector to localize faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the prediction
        confidence = detections[0, 0, i, 2]
        # filter out weak detections
        if confidence > DETECT_CONF_THRESH:
            # compute the (x, y)-coordinates of the bounding box for the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Need to convert because the face_recognition.face_encoding() doesn't use coordinate system
            box = [(startY, endX, endY, startX)]
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Compute encoding using face_recognition library
            encodings = np.array(face_recognition.face_encodings(rgb, box))

            # perform classification to recognize the face
            preds = recognizer.predict_proba(encodings)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]

            # draw the bounding box of the face along with the associated probability
            text_1 = "True: {} Predicted: {}: {:.2f}%".format(true_name, name, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(image, (startX, startY), (endX, endY),
                (0, 0, 255), 2)
            if( true_name == name):
                color = (0, 255, 0)
            else:
                color = (0, 0, 255)
            cv2.putText(image, text_1, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
    # show the output image
    cv2.imshow("Image", image)
    time_2 = time.time()
    print(f"time taken to recognize {random_image} is {time_2-time_1} seconds")
    print(f"True Label: {true_name}, Predicted Label: {name}")
    key = cv2.waitKey(0)
    if key == ord('q'):
        break
    elif key == ord('n'):
        continue
cv2.destroyAllWindows()

Total images in the specified folder:  42
time taken to recognize person_i (17).jpg is 1.1728973388671875 seconds
True Label: person_i, Predicted Label: person_i
time taken to recognize person_b (20).jpg is 0.6737680435180664 seconds
True Label: person_b, Predicted Label: person_b
time taken to recognize person_p (19).jpg is 0.7876532077789307 seconds
True Label: person_p, Predicted Label: person_p
time taken to recognize person_b (19).jpg is 0.7342276573181152 seconds
True Label: person_b, Predicted Label: person_b
time taken to recognize person_e (22).jpg is 0.6996269226074219 seconds
True Label: person_e, Predicted Label: person_e
time taken to recognize person_n (13).jpg is 0.6777360439300537 seconds
True Label: person_n, Predicted Label: person_n
time taken to recognize person_o (17).jpg is 0.6872448921203613 seconds
True Label: person_o, Predicted Label: person_o
time taken to recognize person_t (22).jpg is 0.7766685485839844 seconds
True Label: person_t, Predicted Label: person_